<a href="https://colab.research.google.com/github/drbilal216/AI_201/blob/main/langchain_function_tool_calling_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

In [ ]:
!pip install langchain langchain_community langchain_google_genai

In [2]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

# just checking
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-523a0e6d-617e-4a1e-a57e-4cef75f6f58d-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [5]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

tools = [add, multiply]
# the model can choose whether to return one tool call, multiple tool calls, or no tool calls at all
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# tools_list=[multiply]
# model_with_tools = llm.bind_tools(tools_list)

In [6]:
result = llm_with_tools.invoke("Hey how are you!")
result

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1e6cb50b-d7c2-47be-82ae-0848959c6987-0', usage_metadata={'input_tokens': 57, 'output_tokens': 17, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

In [7]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-629b1dbc-0acf-4e0b-bfb6-79bbe6ee83b5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'ed8d1025-c6a2-4835-9cea-3b2ded378fce', 'type': 'tool_call'}], usage_metadata={'input_tokens': 61, 'output_tokens': 3, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})

In [8]:
# Forcing a tool call
# Some models support a tool_choice

# If we wanted our model to always call the multiply tool we could do:
always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")

query = "What is 3 * 12"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': '7797bf35-bd92-40aa-9ca2-72eccf751b22',
  'type': 'tool_call'}]

In [9]:

# And if we wanted it to always call at least one of add or multiply, we could do:
always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")

query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3.0, 'b': 12.0},
  'id': 'ce47968f-7999-4684-93e3-125d79e2963f',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11.0, 'b': 49.0},
  'id': '114d5c67-12cc-40f7-95cc-de2a595239d5',
  'type': 'tool_call'}]

### tool calling not completed

### Agents ??

In [ ]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a + b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)


In [ ]:
# Query the agent
response1 = agent.run("What is 10 + 20?")  # Uses the tool
print(response1)



<ipython-input-11-4661abfb96ad>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = agent.run("What is 10 + 20?")  # Uses the tool




> Entering new AgentExecutor chain...
Question: What is 10 + 20?
Thought: I need to add 10 and 20. I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 10 20
Observation: The result is: 30
Thought:Question: What is 10 + 20?
Thought: I need to add 10 and 20. I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 10 20
Observation: The result is: 30
Thought:Question: What is 10 + 20?
Thought: I need to add 10 and 20. I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 10 20
Observation: The result is: 30
Thought:Question: What is 10 + 20?
Thought: I need to add 10 and 20. I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 10 20
Observation: The result is: 30
Thought:Question: What is 10 + 20?
Thought: I need to add 10 and 20. I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 10 20
Observation: The result is: 30
Thought:Question: What is 10 + 20?
Thought: I need to add 10 and 20. I can use